# Read PDF information with LLMs
### let's test how good it the LLMs at extracting information from pdfs

In [2]:
packages = ['pymongo', 'gridfs', 'python-dotenv', 'openai', 'pytesseract', 'pdf2image' ]
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

from pymongo import MongoClient
import os, sys
parent_dir = os.path.abspath('..')
# the parent_dir could already be there if the kernel was not restarted,
# and we run this cell again
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from functions.GridFsFileOperations import GridFsFileOperations

endpoint = '10.0.10.5:27017' # use vpn
#endpoint = '192.168.1.10:27017'
#endpoint = '192.168.229.55:27017'
database = 'supercias'
collection = 'companies'
pdfs_path = '../storage/pdfs/'
# make dir path if it does not exist
if not os.path.exists(pdfs_path):
    os.makedirs(pdfs_path)

# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]

# pdf operator
ops = GridFsFileOperations(db, collection, pdfs_path)

# get collection
collection = db[collection]

#find pdf file
cursor = collection.find()
# get the third company in the collection
comp = cursor[480]
print(comp)

# get all the file name fromt he company 
docs = comp['Documentos']
pdf_names = []
#if company has documents
for value in docs.values():
    if isinstance(value, list) and len(value) > 0:
        [ pdf_names.append(v) for v in value ]


{'_id': ObjectId('66ee3437cda6c02f8334607c'), 'name': 'INMOBILIARIA GALILEA-ECUADOR S.A.', 'id': 714180, 'ruc': '0993052973001', 'Documentos': {'Generales': ['0993052973001_DocumentosGenerales_Oficio Transferencia Acciones_2023-07-10.pdf', '0993052973001_DocumentosGenerales_Oficio Nombramiento Administradores_2018-05-29_HOLZAPFEL DE LA FUENTE MATIAS_GERENTE GENERAL.pdf', '0993052973001_DocumentosGenerales_Oficio Nombramiento Administradores_2018-07-26_HOLZAPFEL DE LA FUENTE JOAQUIN_ADMINISTRADOR TIPO B.pdf', '0993052973001_DocumentosGenerales_Oficio Nombramiento Administradores_2023-05-04_HOLZAPFEL DE LA FUENTE MATIAS_GERENTE GENERAL.pdf', '0993052973001_DocumentosGenerales_Oficio Nombramiento Administradores_2023-08-09_DE LA FUENTE ILLANES VICTOR LUIS_ADMINISTRADOR TIPO B.pdf', '0993052973001_DocumentosGenerales_Oficio Nombramiento Administradores_2023-11-22_GALILEA VIAL FELIPE_ADMINISTRADOR TIPO A.pdf', '0993052973001_DocumentosGenerales_Oficio Nombramiento Administradores_2024-01-19

Download an pdf file to make sure that it is working

Let's download and write all files from mongo

In [2]:
for filename in pdf_names:
    # write file to disk
    ops.write_to_disk(filename)
    print(f'{filename} written to disk')

1793173071001_DocumentosGenerales_Formulario de actualización de datos_2021-10-25.pdf written to disk
1793173071001_DocumentosGenerales_Formulario de actualización de datos_2022-06-26.pdf written to disk
1793173071001_DocumentosGenerales_Razón de inscripción societaria nombramiento_2021-08-04_SANCHEZ ESPINOSA DE LOS MONTEROS ADRIANA MARIA_GERENTE GENERAL.pdf written to disk
1793173071001_DocumentosGenerales_Nombramiento SAS_2021-08-04_SANCHEZ ESPINOSA DE LOS MONTEROS ADRIANA MARIA_GERENTE GENERAL.pdf written to disk
1793173071001_DocumentosGenerales_Formulario de actualización de datos_2022-01-26.pdf written to disk
1793173071001_DocumentosGenerales_Razón de inscripción societaria nombramiento_2021-08-04_ESPINOSA DE LOS MONTEROS ROSERO SARA MARIA_PRESIDENTE.pdf written to disk
1793173071001_DocumentosGenerales_Nombramiento SAS_2021-08-04_ESPINOSA DE LOS MONTEROS ROSERO SARA MARIA_PRESIDENTE.pdf written to disk
1793173071001_DocumentosJuridicos_CONSTITUCION ELECTRONICA SAS_Razón de insc

In [3]:
import os
usb_path = '/run/media/terac/KINGSTON/'
dir_path = usb_path+comp['name']+'/'

# make a directory with the name of the company
#if not os.path.exists(dir_path):
    #os.makedirs(dir_path)

# Rest of the code
#for filename in pdf_names:
    #move(pdfs_path+filename, usb_path+comp['name']+'/'+filename)

In [4]:
from functions.parse_supercias_forms import parse_pdf

# Example usage
filename = pdfs_path + pdf_names[2]
print(filename)
financial_data = parse_pdf(filename)
print(financial_data)


../storage/pdfs/1793173071001_DocumentosGenerales_Razón de inscripción societaria nombramiento_2021-08-04_SANCHEZ ESPINOSA DE LOS MONTEROS ADRIANA MARIA_GERENTE GENERAL.pdf
None


In [5]:
# delete file from disk
#for filename in pdf_names:
    # delete file from disk
    #ops.delete_from_disk(filename)
    #print(f'{filename} deleted from disk')

Make sure that the the openai api is workig correcly

In [6]:
from functions.prompt_openai import test_openai
print(test_openai())

True


uploading the pdf dirrecly to openai is not possible. it is not able to extract the data as of today (2024-09-02). 

So we need to extract the text first.

In [7]:
from functions.extract_text_from_pdf import extract_text_from_pdf

extracted_text = extract_text_from_pdf(filename)
print(extracted_text)

SUPERINTENDENCIA

JAS, VALORES Y SEGUROS



Now ask the summerize correcly the infomation extracted from the extracted text

In [8]:
from functions.prompt_openai import prompt_acta_de_la_junta_general
# gethhh all the information from the files that have the string 'Acta de la junta General'
# filter filenames
pdfs = [ pdf for pdf in pdf_names if 'Acta de la Junta General' in pdf ]

# for pdf in pdfs:
    #filename = pdfs_path + pdf
    #print(filename)
    #extracted_text = extract_text_from_pdf(filename)
    #data = prompt_acta_de_la_junta_general(extracted_text)
    #print(data)

In [10]:
from functions.parse_supercias_forms import parse_pdf
# gethhh all the information from the files that have the string 'Acta de la junta General'
# filter filenames
pdfs = [ pdf for pdf in pdf_names if 'Balance  Estado de Situación Financiera' in pdf ]

for pdf in pdfs:
    filename = pdfs_path + pdf
    print(filename)
    extracted_data = parse_pdf(filename)
    print(extracted_data)

../storage/pdfs/1793173071001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2021-12-31.pdf


now let iterate trhough every document to see if we can get a infomration fo the company as a whole

In [ ]:
''' 
analysis = db['analysis']
# for every pdf file in company
for pdf in pdf_names:
    try:
        # read the file
        write_to_disk(pdf)
        print(f'processing {pdf}...')
        # extract text
        text = extract_text_from_image_pdf(pdf)
        if(len(text) > 16385):
            # too big for openai
            print('text too big for openai')
            continue
        # intepret the data with the llm
        response = prompt_openai({
            "role": "system", 
            "content": "Give me all the information you can aquire from this text. give it in json file " + pdf + ": \n" + text, 
        })
        print('response:', response)
        # save the data
        analysis.insert_one({ 'filename': pdf, 'text_extracted': text, 'interpretation': response })
    # catch any errors
    except Exception as e:
        print(f'Error processing {pdf}: {e}')
'''


' \nanalysis = db[\'analysis\']\n# for every pdf file in company\nfor pdf in pdf_names:\n    try:\n        # read the file\n        write_to_disk(pdf)\n        print(f\'processing {pdf}...\')\n        # extract text\n        text = extract_text_from_image_pdf(pdf)\n        if(len(text) > 16385):\n            # too big for openai\n            print(\'text too big for openai\')\n            continue\n        # intepret the data with the llm\n        response = prompt_openai({\n            "role": "system", \n            "content": "Give me all the information you can aquire from this text. give it in json file " + pdf + ": \n" + text, \n        })\n        print(\'response:\', response)\n        # save the data\n        analysis.insert_one({ \'filename\': pdf, \'text_extracted\': text, \'interpretation\': response })\n    # catch any errors\n    except Exception as e:\n        print(f\'Error processing {pdf}: {e}\')\n'